In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import and read charity_data.csv
import pandas as pd 
application_df = pd.read_csv('Resources/charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop non-beneficial ID columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])

In [3]:
# APPLICATION_TYPE value counts for binning
application_values = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than 1500
replace_application = application_values[application_values < 1500].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app, 'Other')

In [4]:
# CLASSIFICATION value counts for binning
classification_values = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than 1500
replace_class = classification_values[classification_values < 1500].index.tolist()

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls, 'Other')

In [5]:
# ASK_AMT bins
ask_values_0 = application_df['ASK_AMT'][application_df['ASK_AMT'] == 5000]
ask_values_1 = application_df['ASK_AMT'][(application_df['ASK_AMT'] > 5000)]

# Replace in dataframe
for amt in ask_values_0:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '5000')
for amt in ask_values_1:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(amt, '>5000')

In [6]:
# AFFILIATION value counts for binning
affiliation_values = application_df['AFFILIATION'].value_counts()

# Determine which values to replace if counts are less than 1500
replace_affiliation = affiliation_values[affiliation_values < 1500].index.tolist()

# Replace in dataframe
for aff in replace_affiliation:
    application_df.AFFILIATION = application_df.AFFILIATION.replace(aff, 'Other')

In [7]:
# ORGANIZATION value counts for binning
organization_values = application_df['ORGANIZATION'].value_counts()

# Determine which values to replace if counts are less than 1500
replace_organization = organization_values[organization_values < 1500].index.tolist()

# Replace in dataframe
for org in replace_organization:
    application_df.ORGANIZATION = application_df.ORGANIZATION.replace(org, 'Other')

In [8]:
# Generate the categorical variable list
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)

/Users/topeadeolatunji/opt/anaconda3/envs/mlenv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)

/var/folders/lt/c6r0brd14djbk6h6mpz3cz0w0000gn/T/ipykernel_66667/75781698.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(application_cat, 1)


In [11]:
# Split the preprocessed data into features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

/var/folders/lt/c6r0brd14djbk6h6mpz3cz0w0000gn/T/ipykernel_66667/942049030.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df.drop(['IS_SUCCESSFUL'], 1).values


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 18

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hiZdden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               3840      
                                                                 
 dense_1 (Dense)             (None, 50)                6050      
                                                                 
 dense_2 (Dense)             (None, 18)                918       
                                                                 
 dense_3 (Dense)             (None, 1)                 19        
                                                                 
Total params: 10,827
Trainable params: 10,827
Non-trainable params: 0
_________________________________________________________________


2022-10-05 00:01:53.334519: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('optimized_checkpoints/', exist_ok=True)
checkpoint_path = 'optimized_checkpoints/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=4000)

In [16]:
# Train the model
fit_model = nn.fit(X_train, y_train, epochs=150, callbacks=[cp_callback])

Epoch 1/150
804/804 [==============================] - 1s 805us/step - loss: 0.6145 - accuracy: 0.6802
Epoch 2/150
804/804 [==============================] - 1s 778us/step - loss: 0.5861 - accuracy: 0.7127
Epoch 3/150
804/804 [==============================] - 1s 777us/step - loss: 0.5786 - accuracy: 0.7180
Epoch 4/150
804/804 [==============================] - 1s 774us/step - loss: 0.5747 - accuracy: 0.7212
Epoch 5/150
774/804 [===========================>..] - ETA: 0s - loss: 0.5731 - accuracy: 0.7218
Epoch 5: saving model to optimized_checkpoints/weights.05.hdf5
804/804 [==============================] - 1s 795us/step - loss: 0.5723 - accuracy: 0.7223
Epoch 6/150
804/804 [==============================] - 1s 788us/step - loss: 0.5703 - accuracy: 0.7210
Epoch 7/150
804/804 [==============================] - 1s 792us/step - loss: 0.5690 - accuracy: 0.7219
Epoch 8/150
804/804 [==============================] - 1s 860us/step - loss: 0.5685 - accuracy: 0.7218
Epoch 9/150
804/804 [=======

804/804 [==============================] - 1s 788us/step - loss: 0.5557 - accuracy: 0.7291
Epoch 63/150
804/804 [==============================] - 1s 801us/step - loss: 0.5559 - accuracy: 0.7289
Epoch 64/150
804/804 [==============================] - 1s 806us/step - loss: 0.5556 - accuracy: 0.7287
Epoch 65/150
516/804 [==================>...........] - ETA: 0s - loss: 0.5540 - accuracy: 0.7313
Epoch 65: saving model to optimized_checkpoints/weights.65.hdf5
804/804 [==============================] - 1s 804us/step - loss: 0.5556 - accuracy: 0.7293
Epoch 66/150
804/804 [==============================] - 1s 787us/step - loss: 0.5553 - accuracy: 0.7294
Epoch 67/150
804/804 [==============================] - 1s 788us/step - loss: 0.5556 - accuracy: 0.7292
Epoch 68/150
804/804 [==============================] - 1s 791us/step - loss: 0.5552 - accuracy: 0.7290
Epoch 69/150
804/804 [==============================] - 1s 783us/step - loss: 0.5550 - accuracy: 0.7295
Epoch 70/150
519/804 [==========

804/804 [==============================] - 1s 811us/step - loss: 0.5514 - accuracy: 0.7319
Epoch 124/150
804/804 [==============================] - 1s 825us/step - loss: 0.5517 - accuracy: 0.7324
Epoch 125/150
265/804 [========>.....................] - ETA: 0s - loss: 0.5579 - accuracy: 0.7242
Epoch 125: saving model to optimized_checkpoints/weights.125.hdf5
804/804 [==============================] - 1s 788us/step - loss: 0.5514 - accuracy: 0.7324
Epoch 126/150
804/804 [==============================] - 1s 788us/step - loss: 0.5510 - accuracy: 0.7321
Epoch 127/150
804/804 [==============================] - 1s 779us/step - loss: 0.5515 - accuracy: 0.7323
Epoch 128/150
804/804 [==============================] - 1s 778us/step - loss: 0.5517 - accuracy: 0.7322
Epoch 129/150
804/804 [==============================] - 1s 757us/step - loss: 0.5515 - accuracy: 0.7318
Epoch 130/150
263/804 [========>.....................] - ETA: 0s - loss: 0.5516 - accuracy: 0.7337
Epoch 130: saving model to op

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.8546 - accuracy: 0.6556 - 211ms/epoch - 789us/step
Loss: 0.8546350002288818, Accuracy: 0.6556268334388733


In [18]:
# Export the model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')